# Script for Class #4

In [ ]:
import numpy as np
from scipy import linalg as la
from scipy import stats as st
from scipy import optimize as opt
from matplotlib import pyplot as plt
from time import time

from IPython.display import set_matplotlib_formats
%matplotlib inline
set_matplotlib_formats('svg')
plt.rcParams['figure.figsize'] = [12, 5]

## Solving for the General Equilibrium when Explicit Prices are Involved

In [ ]:
beta = 0.97
gamma = 1.5
y = 1.0
n = 100 + 1
a = np.linspace(-5, 5, num=n)  # ensuring there's a value that is exactly zero, see later

In [ ]:
rSol = 1 / beta - 1
print('r* = {:.5f}'.format(rSol))

In [ ]:
class Agent:
    
    def __init__(self, beta, gamma, a, y):
        self.beta = beta
        self.gamma = gamma
        self.y = y
        self.a = a
        
    def __call__(self, r, tol=1e-6):
        n = self.a.size
        v = np.zeros((n,1))
        v_new = np.zeros((n,1))
        dr = np.zeros((n,1), dtype=int)
        criterion = 1
        n_iter = 0
        t0 = time()
        while criterion > tol:
            n_iter += 1
            for i in range(n):
                c = self.y + self.a[i] * (1 + r) - self.a
                c[c<=0] = np.nan
                u = c ** (1 - self.gamma) / (1 - self.gamma)
                obj = u + self.beta * v[:, -1]
                v_new[i] = np.nanmax( obj )
                dr[i] = obj.tolist().index(v_new[i])
            v = np.block([v, v_new])
            criterion = np.max(np.abs(v[:, -1] - v[:, -2]))
        t1 = time()
        a_opt = self.a[dr]
        self.v = v
        print('VFI took {0:.3f} seconds, {1} iterations (r={2:.3f}%).'.format(t1-t0, n_iter, r*100))
        # c_opt = self.y + self.a * (1 + r) - a_opt
        return a_opt

In [ ]:
rLo, rHi = np.array([0.75, 1.25]) * rSol
ra = Agent(beta, gamma, a, y)

In [ ]:
"fill me in!"

In [ ]:
print('Analytical solution: r = {:.50f}'.format(rSol))
print(' Numerical solution: r = {:.50f}'.format(rStar))

## From Policy Functions to Endogenous Ergodic Distributions

Same problem as before, add uncertainty in $Y_t$

In [ ]:
a_num = 100
a_min = -5
a_max = 5
A = np.linspace(a_min, a_max, num=a_num)
Y = np.array([0.5, 1.5])
Pi = np.array([[0.75, 0.25],
               [0.25, 0.75]])
# Y = np.array([0.25, 1.00, 1.75])
# Pi = np.array([[0.65, 0.25, 0.10],
#                [0.20, 0.60, 0.20],
#                [0.10, 0.25, 0.65]])
beta = 0.97
gamma = 2.0

In [ ]:
def ergodic_distribution(P):
    eigvalues, eigvectors = la.eig(P)
    real_eigvalues, positions = [], []
    for i, l in enumerate(eigvalues):
        if np.imag(l) == 0.0:
            positions.append(i)
            real_eigvalues.append(l)
    real_eigvalues = np.array(real_eigvalues)
    real_eigvectors = np.real( eigvectors[:, positions] )
    unit_eigvalue = np.argmin( np.abs( real_eigvalues - 1 ) )
    ergo_dist = real_eigvectors[:, unit_eigvalue]
    ergo_dist /= ergo_dist.sum()
    return ergo_dist

In [ ]:
def solve_vfi(r, A, Y, beta, gamma, tol=1e-6):
    na = A.size
    ny = Y.size
    V0 = np.zeros((na, ny))
    dr = np.zeros((na, ny), dtype=int)
    crit = 1.0
    n_iter = 0
    t0 = time()
    while crit > tol:
        n_iter += 1
        V1 = np.zeros_like(V0)
        U = np.zeros((na, ny))
        for i in range(na):
            for j in range(ny):
                C = Y[j] + (1 + r) * A[i] - A
                C[C < 0] = np.nan
                U[:, j] = C ** (1 - gamma) / (1 - gamma)
            objective = U + beta * V0 @ Pi.T
            V1[i, :] = np.nanmax(objective, axis=0)
        crit = np.max( np.max( np.abs( V1 - V0 ) ) )
        V0[:] = V1
    t1 = time()
    for i in range(na):
        for j in range(ny):
            C = Y[j] + (1 + r) * A[i] - A
            C[C < 0] = np.nan
            U[:, j] = C ** (1 - gamma) / (1 - gamma)
        objective = U + beta * V0 @ Pi.T
        dr[i, :] = np.nanargmax(objective, axis=0)
    pf_a = A[dr]
    print('VFI solved with r = {0:.10f}%; {1:.3f} seconds'.format(r*100, t1-t0))
    return pf_a

In [ ]:
def market_clearing(r, beta=0.97, gamma=2.0, tol=1e-6, full_output=False):
    na = A.size
    ny = Y.size
    ns = na * ny
    pa = np.zeros((na, na, ny), dtype=int)
    pf_a = solve_vfi(r, A, Y, beta, gamma)
    pass

In [ ]:
rStar, diagnostics = "fill me in!"

In [ ]:
diagnostics

In [ ]:
pass

In [ ]:
ergo_dist = ergo_dist.reshape((Y.size, A.size)).T
marginal_dist_income = ergodic_distribution(Pi)
dist_if_y_lo = ergo_dist[:, 0] / marginal_dist_income[0]
dist_if_y_hi = ergo_dist[:, -1] / marginal_dist_income[-1]

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2)
ax[0].plot(A, A, color='black', alpha=0.5, linestyle='dashed')
ax[0].plot(A, pf_a[:, 0], color='red', linewidth=2, label="$A'(A, Y^l)$")
ax[0].plot(A, pf_a[:, 1], color='green', linewidth=2, label="$A'(A, Y^h)$")
ax[0].legend()
ax[0].set_title('Pol. fun. assets')
ax[0].set_xlabel('$A$')
ax[0].set_ylabel("$A'(A, Y)$")
ax[1].plot(A, dist_if_y_lo, color='red', linewidth=2, label='$\lambda(A | Y^l)$')
ax[1].plot(A, dist_if_y_hi, color='green', linewidth=2, label='$\lambda(A | Y^h)$')
ax[1].legend()
ax[1].set_xlabel('$A$')
ax[1].set_title('$\lambda(A | Y)$')
plt.tight_layout()
plt.show()

The last plot on the right makes it look like some probabilities on the endogenous distribution are negative. They are not. The $y$-axis has been scaled-and-shifted by `1e-14+1e-2`, which means that each tick label $i$ on the vertical axis are $i \times 10^{-14} + 1 \times 10^{-2}$. Therefore, the zero displayed is actually $0.01$. The label $-1$ actually is $0.004 - 10^{-14} > 0$.